In [9]:
import os
import requests

# Define the FastAPI server URL
base_url = "https://localhost:20000"

os.environ["no_proxy"] = "localhost,172.16.87.75"

# 1. Check Server Status
response = requests.get(f"{base_url}/health", verify=False)
if response.status_code == 200:
    print("Server is enabled:", response.json())

# 2. Make a POST request to /generate_accent/liuyifei
digital_data = {
    "vc_name": "user123",
}
# 使用 'files' 上传文件
with open("./vc/liuyifei.wav", "rb") as f:
    files = {"files": f}
    response = requests.post(f"{base_url}/generate_accent/liuyifei", data=digital_data, files=files, verify=False)

if response.status_code == 200:
    print("Generated digital audio:", response.json())
else:
    print("Failed to generate digital audio:", response.text)

# 3. Make a POST request to /generate_tts
preprocess_data = {
    "vc_uid": response.json()["vc_uid"],
    "tts_text": "这是一段测试文本"
}

response = requests.post(f"{base_url}/generate_tts", json=preprocess_data, verify=False)
if response.status_code == 200:
    print("Preprocessing complete:", response.json())
else:
    print("Failed to preprocess video:", response.text)


Server is enabled: {'status': 'ojbk'}
Generated digital audio: {'vc_uid': '78391fc8', 'file_paths': ['vc/78391fc8_liuyifei.wav']}
78391fc8
Preprocessing complete: {'task_id': 'ed2d2ef5'}


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [26]:
!pip install aiohttp pyaudio av aiortc websockets


  Using cached websockets-14.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached websockets-14.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [19]:
# !curl -x "" -X POST https://172.16.87.75:20000/audio -F "file=@vc/liuyifei.wav" -F "sessionid=0"

import aiohttp
import asyncio

async def upload_audio_file(session_id, file_path):
    async with aiohttp.ClientSession() as session:
        with open(file_path, 'rb') as file:
            audio_data = {'file': file}
            payload = {'sessionid': str(session_id)}
            form = aiohttp.FormData()
            form.add_field('sessionid', str(session_id))
            form.add_field('file', open(file_path, 'rb'), filename=file_path)
            
            async with session.post('https://localhost:20000/audio', data=form, ssl=False) as response:
                response_json = await response.json()
                print(f"Response: {response_json}")

def run_async_main(session_id, file_path):
    try:
        # Check if there's an existing event loop and use it
        loop = asyncio.get_running_loop()
        loop.create_task(upload_audio_file(session_id, file_path))
    except RuntimeError:
        # If no running loop, use asyncio.run()
        asyncio.run(upload_audio_file(session_id, file_path))

# Example usage
session_id = 0
file_path = 'vc/liuyifei.wav'
run_async_main(session_id, file_path)

Response: {'session_id': 0, 'code': -1, 'msg': 'Invalid session ID'}


In [40]:
import asyncio
import json
import aiohttp
import pyaudio
import av
from aiortc import RTCPeerConnection, RTCSessionDescription
from aiortc.contrib.media import MediaPlayer
import websockets
import ssl
import os

pc = None

async def send_offer(session, url):
    global pc

    pc = RTCPeerConnection()

    # Add transceivers for video (recvonly) and audio (sendonly)
    pc.addTransceiver('video', direction='recvonly')
    pc.addTransceiver('audio', direction='sendonly')

    # Create an offer and set the local description
    offer = await pc.createOffer()
    await pc.setLocalDescription(offer)

    # Wait for ICE gathering to complete
    await ice_gathering_complete(pc)

    # Send the offer to the /offer endpoint on the server
    async with session.post(f'{url}/offer', json={
        'sdp': pc.localDescription.sdp,
        'type': pc.localDescription.type
    }, ssl=False) as response:
        answer = await response.json()

    # Set the remote description
    sessionid = answer.get('sessionid')
    print(f"Session ID: {sessionid}")
    await pc.setRemoteDescription(RTCSessionDescription(sdp=answer['sdp'], type=answer['type']))

    return sessionid, pc

async def ice_gathering_complete(pc):
    if pc.iceGatheringState == 'complete':
        return

    ice_complete = asyncio.Future()

    @pc.on('icegatheringstatechange')
    def check_ice():
        if pc.iceGatheringState == 'complete':
            ice_complete.set_result(True)

    await ice_complete

async def send_audio_data(websocket, sessionid, audio_data):
    # Encode audio data to base64 string to ensure safe transmission
    await websocket.send(json.dumps({
        'type': 'audio',
        'sessionid': sessionid,
        'data': audio_data.decode('latin-1')  # Convert bytes to string
    }))

async def receive_video(track):
    # Optionally, process the received video frames
    while True:
        frame = await track.recv()
        # Process the video frame (e.g., display, save, etc.)
        # For this example, we will just print a message
        print("Received video frame")

async def main(audio_file_path="vc/liuyifei.wav"):
    url = "https://localhost:20000"  # Replace with your server's address
    ws_url = "wss://localhost:20000/stream"  # WebSocket URL

    async with aiohttp.ClientSession() as session:
        # Establish WebRTC connection
        sessionid, pc = await send_offer(session, url)

        # Set up WebSocket connection
        ssl_context = ssl.create_default_context()
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE
        async with websockets.connect(ws_url, ssl=ssl_context) as websocket:
            # Send session ID to associate with server-side processing
            await websocket.send(json.dumps({'type': 'session', 'sessionid': sessionid}))

            # Set up audio source (microphone or pre-recorded file)
            audio_source = None
            # p = pyaudio.PyAudio()
            # try:
            #     # Try to open the microphone for audio input
            #     stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=1024)
            #     audio_source = 'microphone'
            #     print("Using microphone for audio input")
            # except OSError as e:
            #     print(f"Error opening microphone: {e}")
            #     # No microphone available, use a pre-recorded audio file
            #     audio_source = 'file'
            #     print("Using pre-recorded audio file for audio input")
            #     if not os.path.exists(audio_file_path):
            #         print(f"Audio file not found at {audio_file_path}")
            #         return
            #     audio_player = MediaPlayer(audio_file_path)
            #     audio_stream = audio_player.audio
            
            audio_source = 'file'
            print("Using pre-recorded audio file for audio input")
            if not os.path.exists(audio_file_path):
                print(f"Audio file not found at {audio_file_path}")
                return
            audio_player = MediaPlayer(audio_file_path)
            audio_stream = audio_player.audio

            # Start receiving video
            @pc.on("track")
            async def on_track(track):
                if track.kind == "video":
                    print("Received video track")
                    await receive_video(track)

            try:
                if audio_source == 'microphone':
                    while True:
                        # Read audio data from microphone
                        audio_data = stream.read(1024, exception_on_overflow=False)
                        # Send audio data through WebSocket
                        await send_audio_data(websocket, sessionid, audio_data)
                elif audio_source == 'file':
                    while True:
                        # Read audio frame from the media player
                        frame = await audio_stream.recv()
                        # Get audio data as bytes
                        audio_data = bytes(frame.planes[0])
                        # Send audio data through WebSocket
                        await send_audio_data(websocket, sessionid, audio_data)
                        # Sleep to match the audio frame duration
                        await asyncio.sleep(frame.time_base * frame.samples)
            except KeyboardInterrupt:
                print("Stopping...")
            finally:
                # # Clean up
                # if audio_source == 'microphone':
                #     stream.stop_stream()
                #     stream.close()
                #     p.terminate()
                # elif audio_source == 'file':
                #     audio_player.close()
                await pc.close()
                await websocket.close()

def run_async_main():
    loop = asyncio.get_event_loop()  # 获取当前的 event loop
    if loop.is_running():
        # 如果 event loop 已经在运行，直接调用 main() 函数
        task = loop.create_task(main())
        return task
    else:
        # 如果没有正在运行的 event loop，使用 asyncio.run() 启动任务
        asyncio.run(main())

if __name__ == "__main__":
    run_async_main()



Session ID: d3fe1c23-d2aa-4958-9a14-dc73da0f0f5b
Using pre-recorded audio file for audio input
